## AI services in Fabric (preview)

### Utilizing Azure AI Services in Fabric

Fabric offers two distinct options for leveraging Azure AI services:

#### 1. Pre-built AI Models in Fabric (Preview)

Fabric integrates seamlessly with Azure AI services, enabling you to enhance your data with pre-built AI models without any prerequisites. This is the recommended approach, as you can use your Fabric authentication to access these AI services, with all usage billed against your Fabric capacity. Currently, this option is in public preview, with a limited selection of AI services available.

Available AI services include:
- **Azure OpenAI Service** (Rest API, Python SDK, Synapse ML)
- **Text Analytics** (Rest API, Synapse ML)
- **Azure AI Translator** (Rest API, Synapse ML)

These services are accessible out of the box in Fabric, supporting both RESTful API and SynapseML. You can also use the OpenAI Python Library to access the Azure OpenAI service within Fabric. For more details on the available models, refer to [prebuilt AI models in Fabric](#).

#### 2. Bring Your Own Key (BYOK)

If you prefer to provision your AI services directly on Azure, you can bring your own key (BYOK) to access them from Fabric. This option is ideal if the AI services you need aren't yet supported by the pre-built AI models in Fabric.

For more information on using Azure AI services with BYOK, visit [Azure AI services in SynapseML with bring your own key](#).


**Note:**  
-  Prebuilt AI models are currently available in preview and are offered for free with certain limitations. Specifically, for OpenAI models, the limit is 20 requests per minute per user.
-  Support gpt-35-turbo, gpt-4-32K, text-embedding-ada-002 (version 2)


In [ ]:
import synapse.ml.core
from synapse.ml.services.openai import *

## Chat 

ChatGPT and GPT-4 models are language models that are optimized for conversational interfaces.

In [ ]:
from synapse.ml.services.openai import OpenAIChatCompletion
from pyspark.sql import Row
from pyspark.sql.types import *


def make_message(role, content):
    return Row(role=role, content=content, name=role)


chat_df = spark.createDataFrame(
    [
        (
            [
                make_message(
                    "system", "You are an AI chatbot with red as your favorite color"
                ),
                make_message("user", "Whats your favorite color"),
            ],
        ),
        (
            [
                make_message("system", "You are very excited"),
                make_message("user", "How are you today"),
            ],
        ),
    ]
).toDF("messages")


In [ ]:


chat_completion = (
    OpenAIChatCompletion()
    .setDeploymentName("gpt-35-turbo-0125") # deploymentName could be one of {gpt-35-turbo-0125 or gpt-4-32k}
    .setMessagesCol("messages")
    .setErrorCol("error")
    .setOutputCol("chat_completions")
)

In [ ]:
chat_completion_df = chat_completion.transform(chat_df).select(
        "messages", "chat_completions.choices.message.content"
    )

In [ ]:
display(chat_completion_df)

Output should look something like below

| messages                                                                                                                                                                 | content                                                                                   |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------|
| [{role='system', content='You are an AI chatbot with red as your favorite color', name='system'), Row(role='user', content='Whats your favorite color', name='user'}] | ['My favorite color is red! What about you?']                                             |
| [{role='system', content='You are very excited', name='system'), Row(role='user', content='How are you today', name='user'}]                                          | ["I'm doing great, thank you for asking! I'm feeling really excited today. How are you?"] |

# Embeddings

In [ ]:
from pyspark.sql import Row

data = [
    Row(text='The early bird catches the worm.'),
    Row(text='Knowledge is power.'),
    Row(text='Actions speak louder than words.'),
    Row(text='A picture is worth a thousand words.')
]

# Create DataFrame
df = spark.createDataFrame(data)

In [ ]:
embedding = (
    OpenAIEmbedding()
    .setDeploymentName("text-embedding-ada-002") # set deployment_name as text-embedding-ada-002
    .setTextCol("text")
    .setOutputCol("out")
)


In [ ]:
embedding_df = embedding.transform(df)

In [ ]:
display(embedding_df)

Your output should look something like this. Number of values in array may be different

| text                                                      | OpenAIEmbedding_cb1548908c98_error | out                                                                                                                                                                                        |
|:----------------------------------------------------------|:-----------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| The early bird catches the worm.                          |                                   | {"type":1,"values":[0.004523789,-0.019567443,0.018745321,-0.015432109,-0.013654789,0.004321876,-0.016543210,-0.012345678,-0.006543210,-0.024567890,0.017123456,0.035678901,0.013456789,-0.002345678,-0.001234567,-0.010987654,0.028765432,-0.018765432,0.026543210]} |
| Knowledge is power.                                       |                                   | {"type":1,"values":[0.003654321,-0.021234567,0.016789012,-0.013456789,-0.015432109,0.002345678,-0.014321098,-0.010987654,-0.007654321,-0.026543210,0.015678901,0.033456789,0.011234567,-0.002987654,-0.003456789,-0.009876543,0.027654321,-0.016789012,0.024567890]} |
| Actions speak louder than words.                          |                                   | {"type":1,"values":[0.005678901,-0.020987654,0.017654321,-0.014567890,-0.013456789,0.005432109,-0.017890123,-0.011234567,-0.008765432,-0.023456789,0.018901234,0.034567890,0.013456789,-0.001234567,-0.002345678,-0.011098765,0.028901234,-0.017123456,0.026789012]} |
| A picture is worth a thousand words.                      |                                   | {"type":1,"values":[0.003987654,-0.019876543,0.018012345,-0.014987654,-0.014321098,0.003456789,-0.015678901,-0.012345678,-0.005678901,-0.025678901,0.017890123,0.032345678,0.012345678,-0.001987654,-0.002345678,-0.010123456,0.029876543,-0.016543210,0.024987654]} |
